In [42]:
import numpy as np
import pandas as pd

In [61]:
data = np.array([
    [80, 750, 30, 1, 25, 0],  # Approved
    [40, 600, 50, 1, 45, 1],  # Rejected
    [90, 780, 20, 1, 20, 0],  # Approved
    [35, 550, 60, 0, 50, 1],  # Rejected
    [70, 720, 35, 1, 30, 0]   # Approved
])

X_train = data[:, :-1]   # features
y_train = data[:, -1]    # labels (0=Approved, 1=Rejected)


# 2. Test Dataset (new applicants)

X_test = np.array([
    [60, 710, 40, 1, 35],  # Expected Approved
    [30, 570, 55, 0, 52]   # Expected Rejected
])
y_test = np.array([0, 1])  # ground truth


# 3. Normalize 
X_min, X_max = X_train.min(axis=0), X_train.max(axis=0)
X_train = (X_train - X_min) / (X_max - X_min)
X_test = (X_test - X_min) / (X_max - X_min)


weights = np.array([
    X_train[y_train == 0][0],  # Approved prototype (class 0)
    X_train[y_train == 1][0]   # Rejected prototype (class 1)
])
labels = np.array([0, 1])

# 5. LVQ Training

alpha = 0.3
epochs=10

for epoch in range(epochs):
    for i in range(len(X_train)):
        xi, target = X_train[i], y_train[i]

        # Find nearest prototype
        distances = np.linalg.norm(weights - xi, axis=1)
        winner = np.argmin(distances)

        # Update rule
        if labels[winner] == target:
            weights[winner] += alpha * (xi - weights[winner])
        else:
            weights[winner] -= alpha * (xi - weights[winner])

    alpha *= 0.5  


# 6. Prediction Function

def lvq_predict(x, weights, labels):
    distances = np.linalg.norm(weights - x, axis=1)
    winner = np.argmin(distances)
    return labels[winner]


# 7. Training Set Evaluation

train_predictions = [lvq_predict(x, weights, labels) for x in X_train]
train_predictions = np.array(train_predictions)
train_acc = np.mean(train_predictions == y_train) * 100

print("Training Predictions:", train_predictions)
print("Training True Labels:", y_train)
print(f"Training Accuracy: {train_acc:.2f}%\n")


# 8. Testing Set Evaluation
test_predictions = [lvq_predict(x, weights,labels) for x in X_test]
test_predictions = np.array(test_predictions)
test_acc = np.mean(test_predictions == y_test) * 100

print("Test Predictions:", test_predictions)
print("Test True Labels:", y_test)
print(f"Test Accuracy: {test_acc:.2f}%")


Training Predictions: [0 1 0 1 0]
Training True Labels: [0 1 0 1 0]
Training Accuracy: 100.00%

Test Predictions: [0 1]
Test True Labels: [0 1]
Test Accuracy: 100.00%
